In [1]:
import os
import json
import requests
import datetime

sat_api_url = "https://landsatlook.usgs.gov/sat-api"

In [2]:
min_cloud = 0
max_cloud = 50

date_min="2015-03-01"
date_max="2015-09-30"

# format datatime for query
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")


query = {
    "time": f"{start}/{end}",
    "query": {
        "bbox":[-180,40,180,80],
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"}
    },
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
}

In [3]:
headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip",
        "Accept": "application/geo+json",
    }

url = f"{sat_api_url}/stac/search"
data = requests.post(url, headers=headers, json=query).json()

In [7]:
# this is the maximum possible
scenes = data['meta']['found']
print(f'{scenes} scenes')

144640 scenes


In [9]:
# for all years, estimate
scenes_all = scenes * 5
print(f'{scenes_all} scenes')

723200 scenes


In [15]:
# estimate of 70 MB per band, convert to TB
# scenes * bands * 70 * years
size = scenes_all * 5 * (70/1024)
print(f'{size:,} GB')

247,187.5 GB


In [16]:
# Assuming requesting each file entirely once
cost_request = round((scenes_all / 1000) * 0.0004, 2)
cost_bandwidth = round(size * 0.02, 2)
print(f'${cost_request} ${cost_bandwidth:,}')

$0.29 $4,943.75


In [ ]:
# TODO:
# Limit to path/rows over land, reduce number of months included
# Actually query for the months each year, currently just assumes each year is roughly the same
# Limit to over land only